# Naive Bayes Sentiment Analysis

In [1]:
import os
import sys
import re
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

cur_dir = os.getcwd()
sys.path.append(cur_dir)
print(cur_dir)

/Users/ntphuong/Dropbox/courses/cs579/notebooks/naive_bayes_sentiment_analysis/notebooks


## Load datasets

In [2]:
data_paths = sorted(glob.glob("../datasets/*.txt"))
data_dict = {"sentence": [], "label": []}
data_path = data_paths[2]
data_name = data_path.split("/")[-1][:-4]
print(data_name)
with open(data_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        s = line.strip().split("\t")
        data_dict["sentence"].append(s[0])
        data_dict["label"].append(int(s[1]))
data_df = pd.DataFrame(data_dict)
data_df.head()

yelp


sentence  label
0                           Wow... Loved this place.      1
1                                 Crust is not good.      0
2          Not tasty and the texture was just nasty.      0
3  Stopped by during the late May bank holiday of...      1
4  The selection on the menu was great and so wer...      1

## Task 1: Data Preprocessing

In [3]:
with open("stopwords.txt", "r") as f:
    my_stopwords = [w.strip() for w in f.readlines()]


# Convert to lowercase
def lowercase(sent):
    return sent.lower()


# Remove punctuation
def remove_punctuation(sent):
    return re.sub("[^0-9A-Za-z ]", "", sent)


# Remove stopwords
def remove_stopwords(sent):
    words = [w for w in word_tokenize(sent) if w not in my_stopwords]
#     words = [w for w in word_tokenize(sent) if w not in stopwords.words("english")]
    return " ".join(words)


# Deal with negations
def handle_negation(sent):
    words = []
    negation = ""
    for w in word_tokenize(sent):
        if re.search(r"n[\'o]t", w):
            negation = "not_"
            continue
        elif re.search(r"\W", w):
            negation = ""
        w = negation + w
        words.append(w)
    return " ".join(words)


# Split training/test sets
def split_data(sents, df, data_name):
    train_X, test_X, train_y, test_y = train_test_split(sents, df["label"], test_size=0.2)
    train = pd.DataFrame(data={"sentence": train_X, "label": train_y})
    test = pd.DataFrame(data={"sentence": test_X, "label": test_y})
    train.to_csv(f"../preprocessed_datasets/{data_name}_train.txt")
    test.to_csv(f"../preprocessed_datasets/{data_name}_test.txt")
    return train, test


# Build the bag of word representations
def build_BoW(df, vectorizer=CountVectorizer()):
    X = vectorizer.fit_transform(df["sentence"]).toarray()
    y = df["label"].values
    return X, y

In [4]:
transforms = [
    lowercase,
    handle_negation,
    remove_stopwords,
    remove_punctuation
]
new_sents = []
sents = data_df["sentence"]
for sent in sents:
    new_sent = sent
    for transform in transforms:
        new_sent = transform(new_sent)
    new_sents.append(new_sent.strip())

train, test = split_data(new_sents, data_df, data_name)

## Task 2: Train Multinomial Naïve Bayes

In [5]:
vectorizer = CountVectorizer()
X, y = build_BoW(train, vectorizer)
# The parameter alpha=1.0 is the add-1 laplace smoothing
multinomial_clf = MultinomialNB(alpha=1.0)
multinomial_clf.fit(X, y)

MultinomialNB()

## Task 3: Train Binary Multinominal Naïve Bayes

In [6]:
# The parameter binary=True is to clip all word counts each document at 1
bin_vectorizer = CountVectorizer(binary=True)
X_, y_ = build_BoW(train, bin_vectorizer)
# The parameter alpha=1.0 is the add-1 laplace smoothing
bin_multinomial_clf = MultinomialNB(alpha=1.0)
bin_multinomial_clf.fit(X_, y_)

MultinomialNB()

## Task 4: Evaluation

In [7]:
# Evaluate Multinomial Naïve Bayes
X_test = vectorizer.transform(test["sentence"])
y_test = test["label"].values
y_hat = multinomial_clf.predict(X_test)
print(f"{X.sum()=}")
print(f"{accuracy_score(y_hat, y_test)=:.5f}")

X.sum()=4346
accuracy_score(y_hat, y_test)=0.82500


In [8]:
# Evaluate Binary Multinomial Naïve Bayes
bin_X_test = bin_vectorizer.transform(test["sentence"])
y_test = test["label"].values
bin_y_hat = bin_multinomial_clf.predict(bin_X_test)
print(f"{X_.sum()=}")
print(f"{accuracy_score(bin_y_hat, y_test)=:.5f}")

X_.sum()=4282
accuracy_score(bin_y_hat, y_test)=0.82500


Both Multinomial Naive Bayes and Binary Multinomial Naive Bayes have similar accuracies